In [3]:
import pandas as pd

In [53]:

df_en = pd.read_pickle('English_Headlines_EngFeatures.pkl')
df_tl = pd.read_pickle('Tagalog_Headlines_EngFeatures')

In [54]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_tl['Rating'] = le.fit_transform(df_tl['Rating'].values)
df_en = df_en.sample(frac=1).reset_index(drop=True)
df_tl = df_tl.sample(frac=1).reset_index(drop=True)


In [55]:
df_en['language'] = 'en'
df_tl['language'] = 'tl'

In [56]:
df_en_fake = df_en[df_en['Rating'] == 0][['Statement', 'Rating', 'language']][:30]
df_en_real = df_en[df_en['Rating'] == 1][['Statement', 'Rating', 'language']][:30]
df_tl_fake = df_tl[df_tl['Rating'] == 0][['Statement', 'Rating', 'language']][:30]
df_tl_real = df_tl[df_tl['Rating'] == 1][['Statement', 'Rating', 'language']][:30]

In [57]:
df_sample = pd.concat([df_en_fake, df_tl_fake, df_en_real, df_tl_real], ignore_index=True)

In [58]:
df_sample

,Statement,Rating,language
0,WEF Chauffeur Admits VIP Elites Refuse To Ride...,0,en
1,Avi Silverberg – Bearded biological male power...,0,en
2,Children Of All Ages Welcome To Disneylands LG...,0,en
3,Trans Day of Vengeance canceled,0,en
4,Sending depleted uranium munitions to Ukraine ...,0,en
...,...,...,...
115,Bilang ng mga may food and waterborne disease ...,1,tl
116,DTI Sec Pascual – nakalusot na rin sa Commissi...,1,tl
117,Nakaambang epekto ng El Niño pinatututukan ni ...,1,tl
118,"US, Australian military planes nasa Pinas na p...",1,tl


In [59]:
#df.Rating[:100].value_counts()

In [4]:
df_sample.to_pickle('FULL_Tag_Eng_random_sampled120ROWS.pkl')

NameError: name 'df_sample' is not defined

In [5]:
df = pd.read_pickle('FULL_Tag_Eng_random_sampled120ROWS.pkl')

In [6]:
df.Rating.value_counts()

Rating
0    60
1    60
Name: count, dtype: int64

In [7]:
df.Statement.values


array(['WEF Chauffeur Admits VIP Elites Refuse To Ride In Electric Cars, Demand Gas',
       'Avi Silverberg – Bearded biological male powerlifter smashes women’s record (held by trans person)',
       'Children Of All Ages Welcome To Disneylands LGBTQ ‘Pride Nite’',
       'Trans Day of Vengeance canceled',
       'Sending depleted uranium munitions to Ukraine is a war crime against the Ukrainian people',
       'Vaccine-derived poliovirus outbreak in Burundi and Congo linked to Bill Gates vaccines',
       'YELLEN FIRE IN A CROWDED THEATER: Janet Yellen says only big banks of choice will be bailed out, not regional banks',
       'Merck’s call to lower the age for Gardasil HPV vaccine ‘just another marketing ploy,’ critic says',
       'Teen Cooks Child She Was Babysitting In Oven After Taking ‘Bad Acid’',
       'East Palestine air has higher-than-normal concentrations of 9 toxic chemicals, experts find',
       'ORWELLIAN: Publisher updating Roald Dahl’s British literary classics t

In [8]:
import joblib
from tqdm import tqdm
import pandas as pd

import requests
import whois
import virustotal_python
from requests_html import HTMLSession
from collections import Counter
from urllib.parse import urlparse
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
import time
import datetime
from threading import Thread
import functools
disable_warnings(InsecureRequestWarning)

options = Options()
options.headless = True

from collections import defaultdict
# from urllib.request import urlopen, URLError
# import socket

# class TimeoutException(Exception):
#     def __init__(self, message="Data gathering took too long"):
#         self.message = message
#         super().__init__(self.message)


def timeout(seconds_before_timeout):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            res = [Exception('function [%s] timeout [%s seconds] exceeded!' % (func.__name__, seconds_before_timeout))]
            def newFunc():
                try:
                    res[0] = func(*args, **kwargs)
                except Exception as e:
                    res[0] = e
            t = Thread(target=newFunc)
            t.daemon = True
            try:
                t.start()
                t.join(seconds_before_timeout)
            except Exception as e:
                print('error starting thread')
                raise e
            ret = res[0]
            if isinstance(ret, BaseException):
                raise ret
            return ret
        return wrapper
    return deco



class SitePredictor:
    def __init__(self, timeout_secs=200):
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('headless')
        self.options.add_argument('window-size=1920x1080')
        self.options.add_argument("disable-gpu")

        self.s = Service(ChromeDriverManager().install())

        self.pr_headers = {'API-OPR': '4ok88wgckg8o0cgcswo4gc4kkc0sgocw0woww4o0'}
        self.checker_headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}


        self.session = HTMLSession()
        self.clf = joblib.load('classifiers/XGBoostClassifierNO-VT.joblib')
        self.timeout_sec = datetime.timedelta(seconds=timeout_secs)

    def get_pagerank(self, domain):
        print('pagerank')
        # headers = {'API-OPR':'4ok88wgckg8o0cgcswo4gc4kkc0sgocw0woww4o0'}
        url = 'https://openpagerank.com/api/v1.0/getPageRank?domains%5B0%5D=' + domain
        request = requests.get(url, headers=self.pr_headers)
        if request.status_code == 200:
            result = request.json()
            if result['status_code'] == 200:
                pr_dec = result['response'][0]['page_rank_decimal']
                rank = result['response'][0]['rank']

                return pr_dec, rank

    def whois_checker(self, domain):
        print('whois')
        w = whois.whois(domain)
        if type(w.get('registrar')) == list:
            domain_registrar = w.get('registrar')[0]
        else:
            domain_registrar = w.get('registrar')

        if type(w.get('registrant_postal_code')) == list:
            postal_code = w.get('registrant_postal_code')[0]
        else:
            postal_code = w.get('registrant_postal_code')

        if type(w.get('country')) == list:    
            country = w.get('country')[0]
        else:
            country = w.get('country')
        return domain_registrar, postal_code, country

    # def virus_scan(self, domain):
    #     print('virus')
    #     virus_data = None
    #     while True:
    #         try:
    #             with self.vt_apikey as vtotal:
    #                 resp = vtotal.request(f"domains/{domain}")
    #                 virus_data = resp.json()['data']['attributes']['last_analysis_stats']
    #                 #return virus_data['malicious']
    #         except Exception:
    #             print('restarting in 1 minute-virustotal')    
    #             time.sleep(100)
    #             print('time up')
    #         else:
    #             return virus_data['malicious']        
            
    @timeout(200)
    def link_checker(self, domain):
        print('linkcheck')
       # time.sleep(5)
        r = self.session.get("https://" + domain, verify=False)
        unique_netlocs = Counter(urlparse(link).netloc for link in r.html.absolute_links)
        outbound_n = 0
        local_n = 0

        for link in unique_netlocs:
            # print(link, unique_netlocs[link])
            if domain.lower() in link:
                local_n += unique_netlocs[link]
            else:
                outbound_n += unique_netlocs[link]

        return local_n, local_n + outbound_n

    def check_login(self, driver):
        
        wp_xpath = "//a[starts-with(@href, 'https://wordpress.org')]"
        try:
            displayed = driver.find_element('xpath', wp_xpath).is_displayed()

            return displayed
        except:

            return False

    def check_page_source(self, domain, driver):
        try:
            driver.get(f"https://{domain}/wp-admin")
            wp_pg = "wp-content" in driver.page_source
            return wp_pg
        except:

            return False

    def check_wp(self, domain):
        print('wordpress')
        # service_obj = Service(executable_path="C:/Users/cvaal/chromedriver.exe")
        # driver = webdriver.Chrome(options=options, service=service_obj)
        driver = webdriver.Chrome(service=self.s, options=self.options)

        driver.implicitly_wait(1)
        driver.get(f"https://{domain}/wp-admin")
        login = self.check_login(driver)
        pg_source = self.check_page_source(domain, driver)
        wp = None
        if login == False and pg_source == False:
            wp = False
        else:
            wp = True

        driver.close()
        return wp

    def check_url(self, url):
        print('url check')
        # headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}
        response = requests.get('https://' + url, timeout=5, headers=self.checker_headers)

        return response.status_code

    def get_site_data(self, domain):
        # try:
        # print(check_url(domain))
        # if check_url(domain) == True:
        # try:
        print('gathering data from', domain)
        #print('PLEASE WORK IM BEGGING')
        
        pr_dec, rank = self.get_pagerank(domain)
        domain_registrar, postal_code, country = self.whois_checker(domain)
        #malicious = self.virus_scan(domain)
        local_n, total_links = self.link_checker(domain)
        wp_check = self.check_wp(domain)
        return domain, pr_dec, rank, domain_registrar, postal_code, country, local_n, total_links, wp_check

    def get_and_organize_data(self, domain_list):
        columns = ['Domain', 'Page rank decimal', 'Site Rank', 'Domain registrar', 'Postal code', 'Country of origin', 'No. of Local links', 'Total links', 'Wordpress?']
        site_df_dict = defaultdict(list)
        time_start = datetime.datetime.now()
        for domain in domain_list:
            
            if 'http' in domain:
                domain = urlparse(domain).netloc.strip("www.")
            #print(domain)
            try:
                site_data = self.get_site_data(domain.lower())
                #time_elapsed = datetime.datetime.now() - time_start
                #if time_elapsed > self.timeout_sec:
                    #raise TimeoutException()
                if type(site_data) == tuple:
                    print(site_data)
                    for column, data in zip(columns, site_data):
                        site_df_dict[column].append(data)
                
                else:
                    print('no data gathered')
                    continue

            except Exception as e:
                #print('EXCEPTION')
                print(e)
                continue

                #print(e)
                #rint('site down')
                
        print(site_df_dict)
        return pd.DataFrame(site_df_dict)

    def predict(self, domain_list):
        if domain_list:
            print('scanning sites:', domain_list)
            data = self.get_and_organize_data(domain_list)  # dataframe obj
            #domains = data['Domain'].values
            if isinstance(data, pd.DataFrame) and 'Domain' in data.columns:
                print('got thru')
                print('columns:', data.columns)
                features_df = data.drop('Domain', axis=1)
                print(type(features_df))
                print(features_df)

                preds = self.clf.predict(features_df)
                predictions = [round(value) for value in preds]
                return predictions
            else:
                print('aint got thru')
                return None
        else:
            return None

C:\Users\cvaal\AppData\Local\Temp\ipykernel_43180\4147315713.py:25: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [11]:

# def test(val=1):
#     while True:
#         try:
#             if val == 1:
#                 val = val+1
#             else:
#                 raise TimeoutException
#         except Exception as e:
#             print(e)
#             print('restarting')
#             time.sleep(3)
#             print('time up')
#         else:
#             return val
    

    
# test(1)

In [9]:
import os 
import pickle

In [10]:

text = []
search_res = []
predictions = []
similarity_vals = []
indexes = []
language = []

checkpoint = []

In [14]:
checkpoints = r'C:\Users\cvaal\PycharmProjects\FactAI\checkpoints2'
n_ch = 'checkpoint'
for i in os.listdir(os.path.join(checkpoints, n_ch)):
    with open(os.path.join(os.path.join(checkpoints, n_ch), i), "rb") as input_file:
        print(i)
        checkpoint.append(pickle.load(input_file))



idx.pkl
predictions.pkl
search_res.pkl
similarity_vals.pkl
text.pkl


In [11]:
from image_reader import ImageReader
from google_scraper import GoogleScraper
#from site_predictor import SitePredictor
from similarity_checker import SimilarityChecker

gs = GoogleScraper(n_searches=10)
sp = SitePredictor()

In [18]:
for i in df[-5:].itertuples(index=True, name='Pandas'):
    print(i)

Pandas(Index=115, Statement='Bilang ng mga may food and waterborne disease sa bansa tumaas', Rating=1, language='tl')
Pandas(Index=116, Statement='DTI Sec Pascual – nakalusot na rin sa Commission on Appointments', Rating=1, language='tl')
Pandas(Index=117, Statement='Nakaambang epekto ng El Niño pinatututukan ni Marcos', Rating=1, language='tl')
Pandas(Index=118, Statement='US, Australian military planes nasa Pinas na para sa Balikatan', Rating=1, language='tl')
Pandas(Index=119, Statement='Liberal Party may ieendorsong limang senatoriables sa 2022 elections', Rating=1, language='tl')


In [19]:


texts = []
search_res = []
predictions = []
similarity_vals = []
ratings = []
indexes =  []

count = 0

for row in df.itertuples(index=True, name='Pandas'):
    sc = SimilarityChecker(lang=row.language)
    query = row.Statement
    print(row.Index, query)
    s_results = gs.get_results(query)
    search_title = [title[0] for title in s_results]
    search_links = [title[1] for title in s_results]
    preds = sp.predict(search_links)
    if preds:
        texts.append(query)
        indexes.append(row.Index)
        ratings.append(row.Rating)
        sims = sc.check_similarity(query, search_title)
        search_res.append(s_results)
        predictions.append(preds)
        similarity_vals.append(sims)
        if row.Index % 5 == 0:
            count = count + 1
            checkpoints = r'C:\Users\cvaal\PycharmProjects\FactAI\checkpoints2'
            newpath = os.path.join(checkpoints, f'checkpoint{count}')
            if not os.path.exists(newpath):
                os.makedirs(newpath)

            with open(os.path.join(newpath,'search_res.pkl'), 'wb') as f:
                pickle.dump(search_res, f)
            with open(os.path.join(newpath,'predictions.pkl'), 'wb') as g:
                pickle.dump(predictions, g)
            with open(os.path.join(newpath,'similarity_vals.pkl'), 'wb') as h:
                pickle.dump(similarity_vals, h)
            
            with open(os.path.join(newpath,'text.pkl'), 'wb') as i:
                pickle.dump(text, i)
            with open(os.path.join(newpath,'idx.pkl'), 'wb') as j:
                pickle.dump(indexes, j)


    

        print(s_results)
        print(preds)
        print(sims)

No sentence-transformers model found with name C:\Users\cvaal/.cache\torch\sentence_transformers\jcblaise_roberta-tagalog-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\cvaal/.cache\torch\sentence_transformers\jcblaise_roberta-tagalog-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0 WEF Chauffeur Admits VIP Elites Refuse To Ride In Electric Cars, Demand Gas
searching google the first 10 results of google
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scanning sites: ['https://thepeoplesvoice.tv/wef-chauffeur-admits-vip-elites-refuse-to-ride-in-electric-cars-demand-gas/', 'https://www.westernjournal.com/wef-vip-driver-says-not-allowed-drive-elites-electric-vehicles-reveals/', 'https://www.wnd.com/2023/01/wef-vip-driver-says-not-allowed-drive-elites-electric-vehicles-reveals/', 'https://cleantechnica.com/2023/05/17/wef-sees-huge-drop-in-oil-demand-as-electric-vehicle-sales-rise/', 'https://patriots.win/p/16Zr14Uy5Y/wef-chauffeur-admits-vip-elites-/c/', 'https://mk-mk.facebook.com/people/TRUTH/100063763076669/', 'https://rairfoundation.com/davos-electric-cars-for-commoners-fuel-cars-for-wef-elites/', 'https://en.wikipedia.org/wiki/Electric_vehicle']
gathering data from thep

No sentence-transformers model found with name C:\Users\cvaal/.cache\torch\sentence_transformers\jcblaise_roberta-tagalog-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\cvaal/.cache\torch\sentence_transformers\jcblaise_roberta-tagalog-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1 Avi Silverberg – Bearded biological male powerlifter smashes women’s record (held by trans person)
searching google the first 10 results of google
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scanning sites: ['https://www.foxnews.com/sports/male-powerlifter-breaks-womens-record-held-by-biological-male-protest-transgender-policies#:~:text=A%20male%20powerlifting%20coach%20by,record%20during%20a%20recent%20meet.', 'https://www.foxnews.com/sports/male-powerlifter-breaks-womens-record-held-by-biological-male-protest-transgender-policies#:~:text=A%20male%20powerlifting%20coach%20by,record%20during%20a%20recent%20meet.', 'https://nypost.com/2023/03/30/male-powerlifter-enters-womens-event-breaks-record/', 'https://www.sportskeeda.com/health-and-fitness/man-breaks-trans-woman-powerlifting-record-100-pounds-prove-point#:~:text=Avi

No sentence-transformers model found with name C:\Users\cvaal/.cache\torch\sentence_transformers\jcblaise_roberta-tagalog-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\cvaal/.cache\torch\sentence_transformers\jcblaise_roberta-tagalog-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

117 Nakaambang epekto ng El Niño pinatututukan ni Marcos
searching google the first 10 results of google
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scanning sites: ['https://www.tagalog.com/videos/watch.php?video_id=154997&back=ChannelVideos', 'https://z-upload.facebook.com/abscbnNEWS/posts/10161132550145168', 'https://pia.gov.ph/news/2023/04/19/paghahanda-sa-epekto-ng-el-nino-inutos-ni-pangulong-marcos', 'https://www.bomboradyo.com/pbbm-nanawagan-sa-publiko-na-magtipid-ng-tubig-kuryente-sa-panahon-ng-el-nino', 'https://www.philstar.com/business/stock-commentary/2023/05/22/2268029/marcos-sa-publiko-magtipid-ng-tubig-kuryente-sa-el-nio']
gathering data from tagalog.com
pagerank
whois
linkcheck
wordpress
('tagalog.com', 5.05, '136239', 'ENOM, INC.', 'REDACTED FOR PRIVACY', 'US', 15, 18, False)
gathering data from z-upload.facebook.com
pagerank
whois
linkcheck
wordpress
('z-upload.facebook.com', 4.49, '1647483', 'RegistrarSafe, LLC', '94025', 'US',

No sentence-transformers model found with name C:\Users\cvaal/.cache\torch\sentence_transformers\jcblaise_roberta-tagalog-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\cvaal/.cache\torch\sentence_transformers\jcblaise_roberta-tagalog-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


118 US, Australian military planes nasa Pinas na para sa Balikatan
searching google the first 10 results of google
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scanning sites: ['https://www.philstar.com/pilipino-star-ngayon/bansa/2023/04/09/2257565/us-australian-military-planes-nasa-pinas-na-para-sa-balikatan', 'https://www.youtube.com/watch?v=z5q9mu97YdI', 'https://www.policefilestonite.net/2023/04/09/balikatan-exercises-ng-pinas-us-at-australia-aarangkada-na/', 'https://www.gmanetwork.com/news/topstories/nation/866568/ph-us-kick-off-2023-balikatan-military-exercise/story/', 'https://www.facebook.com/brigadanewsfmnational/videos/larga-brigada-nationwide/3847690165457725/', 'https://tonite.abante.com.ph/2023/04/08/ph-us-australia-balikatan-kasado-na/']
gathering data from philstar.com
pagerank
whois
linkcheck
wordpress
('philstar.com', 5.63, '15414', 'Network Solutions, LLC', '32256', 'US', 201, 277, False

No sentence-transformers model found with name C:\Users\cvaal/.cache\torch\sentence_transformers\jcblaise_roberta-tagalog-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\cvaal/.cache\torch\sentence_transformers\jcblaise_roberta-tagalog-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


119 Liberal Party may ieendorsong limang senatoriables sa 2022 elections
searching google the first 10 results of google
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scraping link
scanning sites: ['https://bandera.inquirer.net/294051/liberal-party-may-ieendorsong-limang-senatoriables-sa-2022-elections', 'https://www.rappler.com/nation/elections/liberal-party-senate-slate-2022-polls/', 'https://www.bworldonline.com/the-nation/2021/09/28/399688/liberal-party-names-initial-candidates-for-2022-polls/', 'https://www.philstar.com/headlines/2021/09/28/2130302/lp-picks-initial-senatorial-bets-2022-polls', 'https://en.wikipedia.org/wiki/2022_Philippine_Senate_election', 'https://legacy.senate.gov.ph/press_release/2021/0930_delima4.asp']
gathering data from bandera.inquirer.net
pagerank
whois
linkcheck
wordpress
('bandera.inquirer.net', 4.91, '445655', 'NETWORK SOLUTIONS, LLC.', '32256', 'US', 31, 32, True)
gathering data from rappler.com
pagerank
whois
lin

In [22]:
import numpy as np

In [41]:
languages = np.array([df.language.values[idx] for idx in indexes])

In [43]:
languages

array(['en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en',
       'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en',
       'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'tl', 'tl', 'tl',
       'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl',
       'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl',
       'tl', 'tl', 'tl', 'tl', 'tl', 'en', 'en', 'en', 'en', 'en', 'en',
       'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en',
       'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en', 'en',
       'en', 'en', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl',
       'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl',
       'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl', 'tl'],
      dtype='<U2')

In [24]:
#text = np.array([df.Statement.values[idx] for idx in indexes])

In [25]:
ratings

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [26]:
np.unique(ratings, return_counts=True)

(array([0, 1]), array([60, 60], dtype=int64))

In [100]:
cleanedList = [[x for x in arr if str(x) != 'nan'] for arr in predictions]

In [101]:
cleanedList

[[0, 0, 0, 1, 1, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
 [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 0, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0],
 [0, 1, 1, 1, 1, 1, 0],
 [0, 1, 1, 0, 1],
 [1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 0, 0],
 [1, 1, 1, 1, 0, 0, 1],
 [1, 1, 1, 0],
 [0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1],
 [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
 [1, 1, 1, 0, 1, 1, 0, 1],
 [1, 0, 0, 1, 0, 1, 1, 1],
 [1, 1, 1, 1, 0, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
 [0, 1, 0, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1],
 [1, 0, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 0],
 [0, 1, 0, 1],
 [0, 1, 0],
 [0, 1, 0, 1],
 [0, 1, 0],
 [0, 0, 0, 0, 0,

In [108]:
predictions_copy = cleanedList

In [110]:
predictions_copy

[[0, 0, 0, 1, 1, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
 [0, 0, 1, 1, 1, 1, 1, 1, 1, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 0, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0],
 [0, 1, 1, 1, 1, 1, 0],
 [0, 1, 1, 0, 1],
 [1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 0, 0],
 [1, 1, 1, 1, 0, 0, 1],
 [1, 1, 1, 0],
 [0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1],
 [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
 [1, 1, 1, 0, 1, 1, 0, 1],
 [1, 0, 0, 1, 0, 1, 1, 1],
 [1, 1, 1, 1, 0, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
 [0, 1, 0, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1],
 [1, 0, 1, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 0],
 [0, 1, 0, 1],
 [0, 1, 0],
 [0, 1, 0, 1],
 [0, 1, 0],
 [0, 0, 0, 0, 0,

In [104]:
max_len = len(min(cleanedList, key=len))
max_len

2

In [111]:
max_len = 15
for i in cleanedList:
    if len(i) < max_len:
        pad = max_len - len(i)
        for j in range(pad):
            i.append(np.nan)

In [142]:
final_preds = [arr[:10] for arr in cleanedList]

In [143]:
preds_padded = [np.array(pred, dtype='float32') for pred in final_preds]

In [144]:
type(preds_padded[0][0])

numpy.float32

In [39]:
with open('search_res.pkl', 'wb') as f:
    pickle.dump(search_res, f)

In [ ]:
#preds = np.array(predictions)

In [ ]:
testarr = np.array([0, 1, 0])

np.pad(testarr, (0, 15), mode='constant', constant_values=5).shape[0]
testarr

array([0, 1, 0])

In [121]:
max_len = len(max(similarity_vals, key=len))
max_len

15

In [54]:

sim_padded = []
max_len = len(max(similarity_vals, key=len))
for i in similarity_vals:
    arrlen = i.shape[0]
    sim_padded.append(np.pad(i, (0, max_len - arrlen), mode='constant', constant_values=np.nan))
sim_padded
    

[array([0.9647811 , 0.7166927 , 0.60091877, 0.4730269 , 0.85992575,
        0.02173234, 0.5838362 , 0.37867093,        nan,        nan,
               nan,        nan,        nan,        nan,        nan],
       dtype=float32),
 array([0.69429016, 0.69429016, 0.6753232 , 0.58485216, 0.8610285 ,
        0.82172966, 0.5604501 , 0.5085802 , 0.82172966, 0.5604501 ,
        0.72415876, 0.82533646, 0.5536964 , 0.77261484,        nan],
       dtype=float32),
 array([0.7584161 , 0.7584161 , 0.9500826 , 0.9500826 , 0.6984921 ,
        0.5884111 , 0.6375831 , 0.6768223 , 0.76139885, 0.70491266,
               nan,        nan,        nan,        nan,        nan],
       dtype=float32),
 array([0.66599166, 0.75562954, 0.86077154, 0.9019486 , 0.7581341 ,
        0.7656458 , 0.6476121 , 0.7855909 , 0.59876263,        nan,
               nan,        nan,        nan,        nan,        nan],
       dtype=float32),
 array([0.7387432 , 0.8113823 , 0.68604004, 0.75870275, 0.78727484,
        0.7224159 , 

In [127]:
sim_padded_cut = [sim[:10] for sim in sim_padded]

(10,)

In [ ]:
texts
search_res
predictions 
similarity_vals
ratings
indexes 

In [147]:
data_dict = {'text': texts, 'similarity values': sim_padded_cut, 'label': ratings, 'predictions': preds_padded, 'language': languages}
df_res = pd.DataFrame(data_dict)
df_res.head()

,text,similarity values,label,predictions,language
0,"WEF Chauffeur Admits VIP Elites Refuse To Ride In Electric Cars, Demand Gas","[0.9647811, 0.7166927, 0.60091877, 0.4730269, 0.85992575, 0.021732343, 0.5838362, 0.37867093, nan, nan]",0,"[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, nan, nan]",en
1,Avi Silverberg – Bearded biological male powerlifter smashes women’s record (held by trans person),"[0.69429016, 0.69429016, 0.6753232, 0.58485216, 0.8610285, 0.82172966, 0.5604501, 0.5085802, 0.82172966, 0.5604501]",0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",en
2,Children Of All Ages Welcome To Disneylands LGBTQ ‘Pride Nite’,"[0.7584161, 0.7584161, 0.9500826, 0.9500826, 0.6984921, 0.5884111, 0.6375831, 0.6768223, 0.76139885, 0.70491266]",0,"[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]",en
3,Trans Day of Vengeance canceled,"[0.66599166, 0.75562954, 0.86077154, 0.9019486, 0.7581341, 0.7656458, 0.6476121, 0.7855909, 0.59876263, nan]",0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, nan]",en
4,Sending depleted uranium munitions to Ukraine is a war crime against the Ukrainian people,"[0.7387432, 0.8113823, 0.68604004, 0.75870275, 0.78727484, 0.7224159, 0.4982155, 0.74112463, 0.62335736, nan]",0,"[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, nan]",en


In [148]:
df_res['sim mean'] = df_res['similarity values'].apply(lambda x: np.nanmean(x))
df_res['pred mean'] = df_res['predictions'].apply(lambda x: np.nanmean(x))

In [149]:

df_res

,text,similarity values,label,predictions,language,sim mean,pred mean
0,"WEF Chauffeur Admits VIP Elites Refuse To Ride In Electric Cars, Demand Gas","[0.9647811, 0.7166927, 0.60091877, 0.4730269, 0.85992575, 0.021732343, 0.5838362, 0.37867093, nan, nan]",0,"[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, nan, nan]",en,0.574948,0.500000
1,Avi Silverberg – Bearded biological male powerlifter smashes women’s record (held by trans person),"[0.69429016, 0.69429016, 0.6753232, 0.58485216, 0.8610285, 0.82172966, 0.5604501, 0.5085802, 0.82172966, 0.5604501]",0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",en,0.678272,1.000000
2,Children Of All Ages Welcome To Disneylands LGBTQ ‘Pride Nite’,"[0.7584161, 0.7584161, 0.9500826, 0.9500826, 0.6984921, 0.5884111, 0.6375831, 0.6768223, 0.76139885, 0.70491266]",0,"[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]",en,0.748462,0.700000
3,Trans Day of Vengeance canceled,"[0.66599166, 0.75562954, 0.86077154, 0.9019486, 0.7581341, 0.7656458, 0.6476121, 0.7855909, 0.59876263, nan]",0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, nan]",en,0.748899,1.000000
4,Sending depleted uranium munitions to Ukraine is a war crime against the Ukrainian people,"[0.7387432, 0.8113823, 0.68604004, 0.75870275, 0.78727484, 0.7224159, 0.4982155, 0.74112463, 0.62335736, nan]",0,"[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, nan]",en,0.707473,0.888889
...,...,...,...,...,...,...,...
115,Bilang ng mga may food and waterborne disease sa bansa tumaas,"[0.86008924, 0.6101923, 0.26533705, 0.23947668, 0.73113847, 0.67479575, 0.58295906, nan, nan, nan]",1,"[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, nan, nan, nan]",tl,0.566284,0.571429
116,DTI Sec Pascual – nakalusot na rin sa Commission on Appointments,"[0.6994511, 0.23830292, 0.56016487, 0.37456146, 0.38632923, 0.5387496, nan, nan, nan, nan]",1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan]",tl,0.466260,0.166667
117,Nakaambang epekto ng El Niño pinatututukan ni Marcos,"[0.92868847, 0.2707497, 0.7628324, 0.6225011, 0.62020695, nan, nan, nan, nan, nan]",1,"[1.0, 0.0, 0.0, 1.0, 1.0, nan, nan, nan, nan, nan]",tl,0.640996,0.600000
118,"US, Australian military planes nasa Pinas na para sa Balikatan","[0.90795505, 0.8013265, 0.4247173, 0.68712807, 0.36387557, 0.7671388, nan, nan, nan, nan]",1,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan]",tl,0.658690,0.166667


In [150]:
df_res['sim mean'].corr(df_res['label'])

0.38810111100440386

In [151]:
df_res['pred mean'].corr(df_res['label'])

-0.0030593698080643964

In [152]:
print('fake news similarity mean:', df_res[df_res['label'] == 0]['sim mean'].mean())
print('real news similarity mean:', df_res[df_res['label'] == 1]['sim mean'].mean())

fake news similarity mean: 0.58752644
real news similarity mean: 0.6749528


In [153]:
print('fake news similarity std:', df_res[df_res['label'] == 0]['sim mean'].std())
print('real news similarity std:', df_res[df_res['label'] == 1]['sim mean'].std())

fake news similarity std: 0.09937383
real news similarity std: 0.10973159


In [154]:
print('fake news similarity skew:', df_res[df_res['label'] == 0]['sim mean'].skew())
print('real news similarity skew:', df_res[df_res['label'] == 1]['sim mean'].skew())

fake news similarity skew: -0.2748174
real news similarity skew: -0.9380851


In [155]:
print('fake news similarity mean:', df_res[df_res['label'] == 0]['pred mean'].mean())
print('real news similarity mean:', df_res[df_res['label'] == 1]['pred mean'].mean())

fake news similarity mean: 0.59278446
real news similarity mean: 0.59103185


In [156]:
print('fake news similarity std:', df_res[df_res['label'] == 0]['pred mean'].std())
print('real news similarity std:', df_res[df_res['label'] == 1]['pred mean'].std())

fake news similarity std: 0.26604855
real news similarity std: 0.30998832


In [157]:
def count_vals(x, nans, ones=True):
    unique, counts = np.unique(x, return_counts=True)
    val_dict = {}
    for i, j in zip(unique, counts):
        if np.isnan(i):
            val_dict['nan'] = j
        else:
            val_dict[i] = j

    if ones == True and nans == False and 1.0 in val_dict.keys():
        return val_dict[1.0]
    elif ones == False and  nans == False and 0.0 in val_dict.keys():
        return val_dict[0.0]
    elif nans == True and ones == False and 'nan' in val_dict.keys():
        return val_dict['nan']

    else:
        return 0

In [158]:
df_res['n real predicted'] = df_res['predictions'].apply(lambda x: count_vals(x, nans=False))
df_res['n fake predicted'] = df_res['predictions'].apply(lambda x: count_vals(x, ones=False, nans=False))
df_res['n nan predicted'] = df_res['predictions'].apply(lambda x: count_vals(x, ones=False, nans=True))

In [159]:
df_res

,text,similarity values,label,predictions,language,sim mean,pred mean,n real predicted,n fake predicted,n nan predicted
0,"WEF Chauffeur Admits VIP Elites Refuse To Ride In Electric Cars, Demand Gas","[0.9647811, 0.7166927, 0.60091877, 0.4730269, 0.85992575, 0.021732343, 0.5838362, 0.37867093, nan, nan]",0,"[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, nan, nan]",en,0.574948,0.500000,4,4,2
1,Avi Silverberg – Bearded biological male powerlifter smashes women’s record (held by trans person),"[0.69429016, 0.69429016, 0.6753232, 0.58485216, 0.8610285, 0.82172966, 0.5604501, 0.5085802, 0.82172966, 0.5604501]",0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",en,0.678272,1.000000,10,0,0
2,Children Of All Ages Welcome To Disneylands LGBTQ ‘Pride Nite’,"[0.7584161, 0.7584161, 0.9500826, 0.9500826, 0.6984921, 0.5884111, 0.6375831, 0.6768223, 0.76139885, 0.70491266]",0,"[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0]",en,0.748462,0.700000,7,3,0
3,Trans Day of Vengeance canceled,"[0.66599166, 0.75562954, 0.86077154, 0.9019486, 0.7581341, 0.7656458, 0.6476121, 0.7855909, 0.59876263, nan]",0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, nan]",en,0.748899,1.000000,9,0,1
4,Sending depleted uranium munitions to Ukraine is a war crime against the Ukrainian people,"[0.7387432, 0.8113823, 0.68604004, 0.75870275, 0.78727484, 0.7224159, 0.4982155, 0.74112463, 0.62335736, nan]",0,"[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, nan]",en,0.707473,0.888889,8,1,1
...,...,...,...,...,...,...,...,...,...,...
115,Bilang ng mga may food and waterborne disease sa bansa tumaas,"[0.86008924, 0.6101923, 0.26533705, 0.23947668, 0.73113847, 0.67479575, 0.58295906, nan, nan, nan]",1,"[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, nan, nan, nan]",tl,0.566284,0.571429,4,3,3
116,DTI Sec Pascual – nakalusot na rin sa Commission on Appointments,"[0.6994511, 0.23830292, 0.56016487, 0.37456146, 0.38632923, 0.5387496, nan, nan, nan, nan]",1,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan]",tl,0.466260,0.166667,1,5,4
117,Nakaambang epekto ng El Niño pinatututukan ni Marcos,"[0.92868847, 0.2707497, 0.7628324, 0.6225011, 0.62020695, nan, nan, nan, nan, nan]",1,"[1.0, 0.0, 0.0, 1.0, 1.0, nan, nan, nan, nan, nan]",tl,0.640996,0.600000,3,2,5
118,"US, Australian military planes nasa Pinas na para sa Balikatan","[0.90795505, 0.8013265, 0.4247173, 0.68712807, 0.36387557, 0.7671388, nan, nan, nan, nan]",1,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan]",tl,0.658690,0.166667,1,5,4


In [160]:
df_res.to_pickle('RES_DF_120SAMPLES_EN_TL.pkl')

In [125]:
type(df_res.iloc[2]['predictions'])

numpy.ndarray

In [134]:
df_res['n nan predicted'].value_counts()

n nan predicted
0    100
Name: count, dtype: int64

In [ ]:
"""
Okay. So, (1) I input a headline into a program, then (2) that program will scrape google for the first 10 results using the inputted headline as a query and get the main titles and the domains of the sites that appeared in the search results, then (3)  the program will compare the meaning or sentiment of the inputted headline to the main titles of the google search result through cosine similarity. This will then return an array containing 10 values, all being floats in-between 0 and 1, 0 meaning  no similarity in meaning and 1 meaning the headline and google search result title have very similar meanings. (4) the program will then use a machine learning model to predict whether the features of the domains of the google search results are in-line with those of fake news sites or real news sites. This will then return another array that contains values in-between 0 and 1, closer to 0 meaning the site is an unreliable source of information, and closer to 1 meaning the site is a reliable source of information. 

So in the end of the process, the program will return two arrays, one array containing 10 values that represent whether the inputted headline's sentiment matches those of the 10 google searches, and the other array, also containing 10 values, that represent whether the domains of the sites that popped up in google's search results, with the headline as the query, have been predicted to be either a reliable source of information or not. 

The main idea behind this program is that if majority of the google search result titles have little similarities in meaning to the inputted headline, while more of the domains the search results come from have been predicted to be mostly reliable sources of information, the inputted headline may possibly be misinformation because the headline is saying a different thing compared to what reliable news sources say. Now, if majority of the google search result titles have little similarities in meaning to the inputted headline, while more of the domains the search results come from have been predicted to be mostly unreliable sources of information, the inputted headline may possibly be telling the truth and is reporting reliable information because the headline is saying a different thing compared to what unreliable news sources say. 

Another way to determine if the inputted headline is reliable or not is if the majority of the google search result titles are similar in meaning to the inputted headline, while the majority of the domains the search results come from have been predicted to be unreliable sources of information, then this would mean that, the inputted headline may possibly be misinformation because the headline is saying the same things to what unreliable news sources say. This works vice versa too, wherein if majority of the google search result titles are similar in meaning to the inputted headline, while majority of the domains the search results come from have been predicted to be mostly reliable sources of information, the inputted headline may possibly be telling the truth and is reporting reliable information because the headline is saying the same things as what reliable news sources say. 
"""

In [14]:
sc.check_similarity('Hindi ako tao', 'Hindi ako tao')

array([1.0000002], dtype=float32)